# TRANSFER LEARNING

## Objetivos

Practicarme con Transfer Learning

Types of Transfer Learning

    "As is" Transfer Learning:
        Using an existing pre-trained model without making any changes.
        For example, using a model trained on ImageNet with its original architecture and weights for predictions on the same set of ImageNet classes.

    Feature Extraction Transfer Learning:
        Using the pre-learned patterns of an existing model (e.g., EfficientNetB0 trained on ImageNet) as feature extractors.
        Typically, adjusting only the output layer for your specific problem, such as changing the number of output classes (e.g., from 1000 ImageNet classes to 10 classes of food).

    Fine-Tuning Transfer Learning:
        Using the pre-learned patterns of an existing model and "fine-tuning" some or all of the underlying layers.
        This involves adjusting the weights of existing layers, often including lower layers, in addition to training new output layers.
        Fine-tuning allows the model to adapt to the specifics of the new task while retaining some knowledge from the pre-trained model.

These distinctions are crucial when deciding how to leverage pre-trained models for your specific machine learning or deep learning task. "As is" transfer learning is useful when the task is similar to the original pre-training task, while feature extraction and fine-tuning are more appropriate for adapting models to new, related tasks.
 


## Librerías necesarias


In [2]:
# que no se impriman info y warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers,callbacks,models,Sequential,losses
import seaborn as sns
from sklearn.metrics import confusion_matrix
import keras_tuner
from tensorflow import keras
from keras import backend as K
import os
import pandas as pd
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub

## Helper Functions

In [4]:

# plot history
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure(figsize=(10,4))
    
    plt.subplot(1,2,1)
    plt.title('Training and validation loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.plot(hist['epoch'], hist['loss'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_loss'],
           label = 'Val Error')
    plt.legend()

    #plt.figure()
    plt.subplot(1,2,2)
    plt.title('Training and validation accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.plot(hist['epoch'], hist['accuracy'],
           label='Train Accuracy')
    plt.plot(hist['epoch'], hist['val_accuracy'],
           label = 'Val Accuracy')
    plt.legend()
    plt.show()

## Carga de datos

In [5]:
seed_value = 57
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [6]:
IMG_SHAPE = (224,224)
BATCH_SIZE = 32

train_dir='/home/marcos/Escritorio/IA/IA---Study/S6 - Transfer Learning - p1/data/Sports-celebrity images/train'
test_dir='/home/marcos/Escritorio/IA/IA---Study/S6 - Transfer Learning - p1/data/Sports-celebrity images/test'

In [7]:
train_datagen = ImageDataGenerator(rescale=1/255.,
                                  validation_split=0.2) # dsp. implementaré data augmentation



#
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = IMG_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = IMG_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)




#
test_datagen = ImageDataGenerator(rescale=1/255.)

test_generator = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMG_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode="categorical")


Found 315 images belonging to 4 classes.
Found 78 images belonging to 4 classes.
Found 50 images belonging to 4 classes.


Probaremos ResNet e ImageNet.


In [8]:
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [9]:
# usaremos tensorboard como callback

# Create TensorBoard callback (functionized beause we need to create a new one for each model)
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  #log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  log_dir = './logs'
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback




In [10]:
# Let's make a create_model() function to create a model from a URL
def create_model(model_url, num_classes=4):
  """
  Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.

  Args:
    model_url (str): A TensorFlow Hub feature extraction URL.
    num_classes (int): Number of output neurons in the output layer,
      should be equal to number of target classes, default 4.
  
  Returns:
    An uncompiled Keras Sequential model with model_url as feature extractor
    layer and Dense output layer with num_classes output neurons.
  """
  # Download the pretrained model and save it as a Keras layer
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False, # freeze the already learned patterns
                                           name="feature_extraction_lyaer",
                                           input_shape=IMG_SHAPE+(3,)) 

  # Create our own model
  model = tf.keras.Sequential([
    feature_extractor_layer,
    layers.Dense(num_classes, activation="softmax", name="output_layer")
  ])
    
  return model  

In [11]:
# RESNET

In [12]:
# Create Resnet model 
resnet_model = create_model(resnet_url,
                            num_classes=4)

In [13]:
# Compile our resnet model
resnet_model.compile(loss="categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [14]:
resnet_history = resnet_model.fit(train_generator,
                                  epochs=5,
                                  steps_per_epoch=len(train_generator),
                                  validation_data=validation_generator,
                                  validation_steps=len(validation_generator),
                                  callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",
                                                                         experiment_name="resnet50V2"
                                                                         )])

Saving TensorBoard log files to: ./logs
Epoch 1/5
10/10 [==============================] - 21s 2s/step - loss: 1.4516 - accuracy: 0.3619 - val_loss: 1.1891 - val_accuracy: 0.5128
Epoch 2/5
10/10 [==============================] - 17s 2s/step - loss: 0.8505 - accuracy: 0.7175 - val_loss: 0.9762 - val_accuracy: 0.6026
Epoch 3/5
10/10 [==============================] - 15s 2s/step - loss: 0.5723 - accuracy: 0.8444 - val_loss: 0.8593 - val_accuracy: 0.6410
Epoch 4/5
10/10 [==============================] - 18s 2s/step - loss: 0.4254 - accuracy: 0.8984 - val_loss: 0.8061 - val_accuracy: 0.6538
Epoch 5/5
10/10 [==============================] - 15s 2s/step - loss: 0.3389 - accuracy: 0.9206 - val_loss: 0.7652 - val_accuracy: 0.6538


In [15]:
scores_resnet = resnet_model.evaluate(test_generator)

2/2 [==============================] - 2s 688ms/step - loss: 0.6969 - accuracy: 0.7000


In [17]:
print(f"Wow!!, en tal sólo 5 epochs logramos una precisión del {round(scores_resnet[1]*100,2)}%")

Wow!!, en tal sólo 5 epochs logramos una precisión del 70.0%


In [18]:
# EFFICIENT NET

In [19]:
# Create EfficinetNetB0 feature extractor model - details on EfficientNet: https://ai.googleblog.com/2019/05/efficientnet-improving-accuracy-and.html#:~:text=EfficientNet%2DB0%20is%20the%20baseline,than%20the%20best%20existing%20CNN.
efficientnet_model = create_model(model_url=efficientnet_url,
                                  num_classes=4)

# Compile EfficientNet model
efficientnet_model.compile(loss="categorical_crossentropy",
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=["accuracy"])

# Fit EfficientNet model to 10% of training data
efficientnet_history = efficientnet_model.fit(train_generator,
                                              epochs=5,
                                              steps_per_epoch=len(train_generator),
                                              validation_data=test_generator,
                                              validation_steps=len(test_generator),
                                              callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",
                                                                                     experiment_name="efficientnetb0")])

Saving TensorBoard log files to: ./logs
Epoch 1/5
10/10 [==============================] - 17s 1s/step - loss: 1.2216 - accuracy: 0.4508 - val_loss: 1.0745 - val_accuracy: 0.6600
Epoch 2/5
10/10 [==============================] - 9s 920ms/step - loss: 0.8632 - accuracy: 0.7683 - val_loss: 0.8688 - val_accuracy: 0.7400
Epoch 3/5
10/10 [==============================] - 9s 927ms/step - loss: 0.6542 - accuracy: 0.8635 - val_loss: 0.7411 - val_accuracy: 0.8200
Epoch 4/5
10/10 [==============================] - 10s 957ms/step - loss: 0.5232 - accuracy: 0.8952 - val_loss: 0.6536 - val_accuracy: 0.8200
Epoch 5/5
10/10 [==============================] - 9s 942ms/step - loss: 0.4360 - accuracy: 0.9238 - val_loss: 0.5976 - val_accuracy: 0.8400


In [20]:
scores_efficientnet = efficientnet_model.evaluate(test_generator)

2/2 [==============================] - 1s 425ms/step - loss: 0.5976 - accuracy: 0.8400


In [22]:
print(f"Wow!!, en tal sólo 5 epochs logramos una precisión del {round(scores_efficientnet[1]*100,2)}%")

Wow!!, en tal sólo 5 epochs logramos una precisión del 84.0%


### Comparemos los modelos usando TensorBoard

Voy hasta en una terminal hasta donde se encuentra la carpeta de logs y ejecuto:

    
   ....$ tensorboard --logdir=./logs
   
   
 (logs es donde guardamos los logs, se especificó más arriba)
 
 Luego, vamos en el navegador a :
 
 http://localhost:6006
 
 
 Veremos algo como lo que se muestra en la imagen siguiente:
 
 <img src='images/TensorBoardf.png'>
 